# Data Grouping

## 📘 Cell 1: Import dan Setup Folder

In [1]:
# 📦 Import library yang dibutuhkan
import os
import pandas as pd

## 📘 Cell 2: Definisikan Folder dan Inisialisasi

In [2]:
# 📁 Path ke folder utama
root_folder = "DataGrouping3"

# 📂 Inisialisasi struktur data untuk semua tipe model
all_data = {
    "ensemble": [],
    "konvensional": [],
    "ann": [],
    "kfold": []
}

## 📘 Cell 3: Membaca Semua CSV dan Normalisasi

In [3]:
# 🔁 Membaca semua file CSV dalam subfolder dan normalisasi kolom
for subdir in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subdir)

    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, file)

            if file.endswith(".csv"):
                df = pd.read_csv(file_path)

                # Tambahkan kolom FileName untuk SEMUA file
                df["FileName"] = file

                # --- Untuk file KFold ---
                if "kfold" in file.lower():
                    df_kfold = df[[
                        "Percobaan", "Model",
                        "Accuracy_Test", "Precision_Test", "Recall_Test", "F1-Score_Test",
                        "FileName"
                    ]].copy()

                    df_kfold = df_kfold.rename(columns={
                        "Accuracy_Test": "Accuracy",
                        "Precision_Test": "Precision",
                        "Recall_Test": "Recall",
                        "F1-Score_Test": "F1-Score"
                    })

                    df_kfold["Tipe"] = "kfold"
                    df_kfold["Sumber"] = subdir
                    all_data["kfold"].append(df_kfold)

                # --- Untuk file ensemble ---
                elif "ensemble" in file.lower():
                    df["Tipe"] = "ensemble"
                    df["Sumber"] = subdir
                    all_data["ensemble"].append(df)

                # --- Untuk file ann ---
                elif "ann" in file.lower():
                    df["Tipe"] = "ann"
                    df["Sumber"] = subdir
                    all_data["ann"].append(df)

                # --- Untuk file model konvensional ---
                elif "konvensional" in file.lower():
                    df["Tipe"] = "konvensional"
                    df["Sumber"] = subdir
                    all_data["konvensional"].append(df)

## 📘 Cell 4: Gabungkan Semua Data

In [4]:
# 📊 Gabungkan semua data menjadi satu DataFrame
df_all = pd.concat(
    all_data["ensemble"] +
    all_data["konvensional"] +
    all_data["ann"] +
    all_data["kfold"],
    ignore_index=True
)

# Pastikan kolom evaluasi numerik
for metric in ["Accuracy", "Precision", "Recall", "F1-Score"]:
    df_all[metric] = pd.to_numeric(df_all[metric], errors="coerce")

## 📘 Cell 5: Fungsi untuk Ambil Model Terbaik/Terburuk

In [5]:
# 🎯 Fungsi untuk mengambil n model terbaik atau terburuk berdasarkan F1-Score
def get_top_bottom(df, tipe=None, top_n=10, worst=False):
    temp = df.copy()
    if tipe:
        temp = temp[temp["Tipe"] == tipe]
    # 🔑 Urutan sorting multi-kriteria
    temp = temp.sort_values(
        ["F1-Score", "Precision", "Recall", "Accuracy"],
        ascending=[worst, worst, worst, worst]
    )
    return temp.head(top_n)

## 📘 Cell 6: Subset Kategori Tambahan

In [6]:
# --- KFold khusus (results_kfold.csv di folder tertentu)
kfold_folders = ["DataSmote", "DataImbalance", "DataOverSamplingRandom", "DataUndersampling"]
df_kfold_special = df_all[
    (df_all["Tipe"] == "kfold") &
    (df_all["Sumber"].isin(kfold_folders)) &
    (df_all["FileName"].str.lower().eq("results_kfold.csv"))
]

In [7]:
# --- Balance (3 folder)
balance_folders = ["DataOverSamplingRandom", "DataSmote", "DataUndersampling"]
df_balance = df_all[df_all["Sumber"].isin(balance_folders)]

In [8]:
# --- PCA (cari kata 'pca' di kolom Percobaan / percobaan)
col_percobaan = [c for c in df_all.columns if c.lower() == "percobaan"]
if col_percobaan:
    pca_col = col_percobaan[0]
    df_pca = df_all[df_all[pca_col].str.lower().str.contains("pca", na=False)]
else:
    df_pca = pd.DataFrame()

In [9]:
# ✅ Normalisasi (normalisasi, normalization, atau pca)
if col_percobaan:
    norm_keywords = r"(normalisasi|normalization|pca)"
    df_norm = df_all[df_all[pca_col].str.lower().str.contains(norm_keywords, na=False)]
else:
    df_norm = pd.DataFrame()

C:\Users\Ryan\AppData\Local\Temp\ipykernel_27028\3703453277.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_norm = df_all[df_all[pca_col].str.lower().str.contains(norm_keywords, na=False)]


## 📘 Cell 7: Ekstrak Data Terbaik & Terburuk

In [10]:
# 🏆 Ekstrak hasil terbaik & terburuk sesuai task
top_20_models = get_top_bottom(df_all, top_n=20)
top_10_ensemble = get_top_bottom(df_all, tipe="ensemble", top_n=10)
top_10_ann = get_top_bottom(df_all, tipe="ann", top_n=10)
bottom_10_ann = get_top_bottom(df_all, tipe="ann", top_n=10, worst=True)   # ✅ Tambahan ANN Terburuk
bottom_20_models = get_top_bottom(df_all, top_n=20, worst=True)
bottom_10_ensemble = get_top_bottom(df_all, tipe="ensemble", top_n=10, worst=True)

# ✅ Tambahan baru
top_10_kfold_special = get_top_bottom(df_kfold_special, top_n=10)
bottom_10_kfold_special = get_top_bottom(df_kfold_special, top_n=10, worst=True)

top_10_balance = get_top_bottom(df_balance, top_n=10)
bottom_10_balance = get_top_bottom(df_balance, top_n=10, worst=True)

top_10_pca = get_top_bottom(df_pca, top_n=10)
bottom_10_pca = get_top_bottom(df_pca, top_n=10, worst=True)

top_10_norm = get_top_bottom(df_norm, top_n=10)           # ✅ Normalisasi Terbaik
bottom_10_norm = get_top_bottom(df_norm, top_n=10, worst=True)  # ✅ Normalisasi Terburuk

## 📘 Cell 8: Simpan ke Excel

In [11]:
# 💾 Simpan semua hasil ke file Excel
output_dir = "ResultGrouping"
os.makedirs(output_dir, exist_ok=True)  # Membuat folder jika belum ada

output_file_full = os.path.join(output_dir, "hasil_klasifikasi_model3_Ryan.xlsx")

with pd.ExcelWriter(output_file_full, engine='xlsxwriter') as writer:
    # sheet lama
    top_20_models.to_excel(writer, sheet_name="20_Model_Terbaik", index=False)
    bottom_20_models.to_excel(writer, sheet_name="20_Model_Terburuk", index=False)
    top_10_ensemble.to_excel(writer, sheet_name="10_Ensemble_Terbaik", index=False)
    bottom_10_ensemble.to_excel(writer, sheet_name="10_Ensemble_Terburuk", index=False)
    top_10_ann.to_excel(writer, sheet_name="10_ANN_Terbaik", index=False)
    bottom_10_ann.to_excel(writer, sheet_name="10_ANN_Terburuk", index=False)
    # sheet tambahan
    top_10_kfold_special.to_excel(writer, sheet_name="10_KFold_Terbaik", index=False)
    bottom_10_kfold_special.to_excel(writer, sheet_name="10_KFold_Terburuk", index=False)
    top_10_balance.to_excel(writer, sheet_name="10_Balance_Terbaik", index=False)
    bottom_10_balance.to_excel(writer, sheet_name="10_Balance_Terburuk", index=False)
    top_10_pca.to_excel(writer, sheet_name="10_PCA_Terbaik", index=False)
    bottom_10_pca.to_excel(writer, sheet_name="10_PCA_Terburuk", index=False)
    top_10_norm.to_excel(writer, sheet_name="10_Normalisasi_Terbaik", index=False)  # ✅ Normalisasi Terbaik
    bottom_10_norm.to_excel(writer, sheet_name="10_Normalisasi_Terburuk", index=False)  # ✅ Normalisasi Terburuk

    workbook = writer.book
    format_full = workbook.add_format({'num_format': '0.000000000000000'})
    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        worksheet.set_column('C:H', 25, format_full)  # kolom metrik & FileName

print(f"File presisi penuh tersimpan: {output_file_full}")

File presisi penuh tersimpan: ResultGrouping\hasil_klasifikasi_model3_Ryan.xlsx


## 📘 Cell 9: Simpan Excel – Ringkas 3 Desimal

In [12]:
output_dir = "ResultGrouping"
os.makedirs(output_dir, exist_ok=True)  # Membuat folder jika belum ada

output_file_round = os.path.join(output_dir, "hasil_klasifikasi_model_ringkas3_Ryan.xlsx")

def round_df(df):
    df_rounded = df.copy()
    for metric in ["Accuracy", "Precision", "Recall", "F1-Score"]:
        if metric in df_rounded.columns:
            df_rounded[metric] = df_rounded[metric].round(3)
    return df_rounded

with pd.ExcelWriter(output_file_round, engine="xlsxwriter") as writer:
    # sheet lama
    round_df(top_20_models).to_excel(writer, sheet_name="20_Model_Terbaik", index=False)
    round_df(bottom_20_models).to_excel(writer, sheet_name="20_Model_Terburuk", index=False)
    round_df(top_10_ensemble).to_excel(writer, sheet_name="10_Ensemble_Terbaik", index=False)
    round_df(bottom_10_ensemble).to_excel(writer, sheet_name="10_Ensemble_Terburuk", index=False)
    round_df(top_10_ann).to_excel(writer, sheet_name="10_ANN_Terbaik", index=False)
    round_df(bottom_10_ann).to_excel(writer, sheet_name="10_ANN_Terburuk", index=False)
    # sheet tambahan
    round_df(top_10_kfold_special).to_excel(writer, sheet_name="10_KFold_Terbaik", index=False)
    round_df(bottom_10_kfold_special).to_excel(writer, sheet_name="10_KFold_Terburuk", index=False)
    round_df(top_10_balance).to_excel(writer, sheet_name="10_Balance_Terbaik", index=False)
    round_df(bottom_10_balance).to_excel(writer, sheet_name="10_Balance_Terburuk", index=False)
    round_df(top_10_pca).to_excel(writer, sheet_name="10_PCA_Terbaik", index=False)
    round_df(bottom_10_pca).to_excel(writer, sheet_name="10_PCA_Terburuk", index=False)
    round_df(top_10_norm).to_excel(writer, sheet_name="10_Normalisasi_Terbaik", index=False)  # ✅ Normalisasi Terbaik
    round_df(bottom_10_norm).to_excel(writer, sheet_name="10_Normalisasi_Terburuk", index=False)  # ✅ Normalisasi Terburuk

    workbook = writer.book
    format_3 = workbook.add_format({'num_format': '0.000'})
    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        worksheet.set_column('C:H', 25, format_3)

print(f"✅ Disimpan versi ringkas ke: {output_file_round}")


✅ Disimpan versi ringkas ke: ResultGrouping\hasil_klasifikasi_model_ringkas3_Ryan.xlsx
